In [ ]:
%load_ext autoreload
%autoreload 2

import os
import igl
import math
import json
import numpy as np
from tqdm import tqdm

from src import tools

## Traverse Data

In [ ]:
data_root = "data/WeFish/converted/"

file_paths = []
for mesh_name in filter(lambda x: x.endswith(".stl"), os.listdir(data_root)):
    file_paths.append(os.path.join(data_root, mesh_name))

print(f"Found {len(file_paths)} files")

In [ ]:
def sample_points_around_surface(n_points, verts, faces, sigma=.1):
    if n_points == 0:
        return np.zeros((0, 3)), np.zeros((0))
    b_coors, faces_ids = igl.random_points_on_mesh(n_points, verts, faces)
    triangles_coords = verts[faces[faces_ids]]
    sampled_points = np.sum(b_coors[:,:,None] * triangles_coords, axis=1) # some over vertices in the triangle
    noise = np.random.normal(scale=sigma, size=sampled_points.shape)
    sampled_points += noise
    return sampled_points

def sample_points_bbox(n_points, bbox_scale=1.):
    if n_points == 0:
        return np.zeros((0, 3)), np.zeros((0))

    points = (np.random.random((n_points, 3)) * 2 - 1) * bbox_scale
    return points

def generate_sdf(verts, faces, bbox_scale=[1., .5, .5], sigma=.1, n_points_surf=80000, n_points_bbox=20000):
    points_1 = sample_points_around_surface(n_points_surf, verts, faces, sigma=sigma)
    points_2 = sample_points_bbox(n_points_bbox, bbox_scale=bbox_scale)
    sampled_points = np.concatenate([points_1, points_2], axis=0)

    '''
        Distance Types:
          igl::SIGNED_DISTANCE_TYPE_PSEUDONORMAL 
          igl::SIGNED_DISTANCE_TYPE_WINDING_NUMBER
          igl::SIGNED_DISTANCE_TYPE_DEFAULT       
          igl::SIGNED_DISTANCE_TYPE_UNSIGNED      
          igl::SIGNED_DISTANCE_TYPE_FAST_WINDING_NUMBER 
    '''
    sdfs, _, _ = igl.signed_distance(sampled_points, verts, faces, sign_type=4)
    return sampled_points, sdfs

# Generate all SDFs

In [ ]:
exp_name = "wefish_1"
save_root = os.path.join("data/sdf", exp_name)

if not os.path.exists(save_root):
    os.mkdir(save_root) 
    os.mkdir(os.path.join(save_root, "samples"))

mapping = {}
saved_names = []    
for idx, fp in enumerate(tqdm(file_paths)):
    verts,_,_,faces,_,_ = igl.read_obj(fp)
    sampled_points, sdfs = generate_sdf(verts, faces)

    data = np.concatenate([sampled_points, sdfs[:, None]], axis=1)

    output_name = f"{idx:04d}"
    np.savez(os.path.join(save_root, "samples", output_name + ".npz"), 
             pos=data[data[:, 3] >= 0],
             neg=data[data[:, 3] < 0]
            )
    
    mapping[output_name] = fp
    saved_names.append(output_name)

with open(os.path.join(save_root, "mapping.json"), "w") as f:
    json.dump(mapping, f)

with open(os.path.join(save_root, "train_split.json"), "w") as f:
    json.dump({exp_name: {"samples":saved_names}}, f)

In [ ]:
# Compress for export
! zip -r converted.zip {save_root}

# Visualize SDFs

In [ ]:
for shape_idx in np.random.choice(len(file_paths), 2):
    verts,_,_,faces,_,_ = igl.read_obj(file_paths[shape_idx])
    sampled_points, sdfs = generate_sdf(verts, faces)

    traces = []
    traces.append(tools.plot_3d_mesh(verts, faces))

    filt = np.abs(sdfs) < 1e-3
    traces.append(tools.plot_3d_point_cloud(sampled_points[filt], fields=sdfs[filt], n_subsample=2000))
    
    filt = sdfs < 0.
    traces.append(tools.plot_3d_point_cloud(sampled_points[filt], fields=sdfs[filt], n_subsample=2000))
    
    traces.append(tools.plot_3d_point_cloud(sampled_points, fields=sdfs, n_subsample=2000))

    tools.show_grid(*traces)

## Training

In [ ]:
# Start Training

#!CUDA_VISIBLE_DEVICES=5 python external/MeshSDF/train_deep_sdf.py -e runs/wolfish_e32

In [ ]:
import os
import torch
import plotly.graph_objects as go

def load_logs(experiment_directory):

    full_filename = os.path.join(experiment_directory, "Logs.pth")

    if not os.path.isfile(full_filename):
        raise Exception('log file "{}" does not exist'.format(full_filename))

    data = torch.load(full_filename)

    return (
        data["loss"],
        data["epoch"],
    )

In [ ]:
run_path = "runs/wolfish_e16/"

loss = load_logs(run_path)
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(loss[1])), y=loss[0]))
fig.show()

In [ ]:
run_path = "runs/wolfish_e32/"

loss = load_logs(run_path)
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(loss[1])), y=loss[0]))
fig.show()

In [ ]:
run_path = "runs/wolfish_e256/"

loss = load_logs(run_path)
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(loss[1])), y=loss[0]))
fig.show()